In [3]:
import numpy as np
from scipy.stats import beta

In [43]:
class BayesianBetaEstimator:
    def __init__(self, prior_alpha=10, prior_beta=10):
        self.alpha = prior_alpha
        self.beta = prior_beta
        self.n = 0  # Keep track of sample size

    def update(self, x):
        if not (0 <= x <= 1):
            raise ValueError("Data points must be in the range [0, 1].")
        self.alpha += x
        self.beta += (1 - x)
        self.n += 1

    def point_estimate(self, scale=2):
        """Estimate the parameters of the data-generating distribution."""
        total = self.alpha + self.beta
        alpha_hat = (self.alpha / total) * scale
        beta_hat = (self.beta / total) * scale
        return alpha_hat, beta_hat

    def get_posterior_params(self):
        """Get raw posterior parameters."""
        return self.alpha, self.beta
    
    def get_params(self, scale=2):
        return self.point_estimate(scale=scale)
    
class OnlineBetaMOM:
    def __init__(self, min_sample_size=20):
        self.n = 0  # Number of observations
        self.mean = 0.0  # Running mean
        self.M2 = 0.0  # Sum of squared differences from the mean (for variance)
        self.min_sample_size = min_sample_size
    
    def update(self, x):
        """
        Update the running mean and variance with a new data point x.
        """
        if x <= 0 or x >= 1:
            raise ValueError("Data points must be in the range (0, 1).")
        
        self.n += 1
        delta = x - self.mean
        self.mean += delta / self.n
        delta2 = x - self.mean
        self.M2 += delta * delta2  # Incremental update for variance

    def get_mean_variance(self):
        """
        Compute the current mean and sample variance.
        """
        if self.n < 2:
            return self.mean, float('nan')  # Variance undefined for n < 2
        sample_variance = self.M2 / (self.n - 1)
        return self.mean, sample_variance

    def estimate_params(self):
        """
        Estimate alpha and beta using the method of moments.
        """
        mean, variance = self.get_mean_variance()
        if self.n < self.min_sample_size or variance <= 0:
            return 1.0, 1.0  # Parameters undefined, choose uniform as default
        
        common_factor = (mean * (1 - mean) / variance) - 1
        alpha = mean * common_factor
        beta = (1 - mean) * common_factor
        return alpha, beta

    def get_params(self):
        return self.estimate_params()

In [70]:
p_values = np.concatenate([np.random.uniform(0, 1, size=3000), np.random.beta(1, 2, size=3000)])

In [71]:
bbe = BayesianBetaEstimator()
bmom = OnlineBetaMOM()
for p in p_values:
    bbe.update(p)
    bmom.update(p)
print(bbe.get_params())
print(bmom.get_params())

(0.8265861237991285, 1.1734138762008715)
(0.9123651518970946, 1.2967298356691772)


In [73]:
bbe = BayesianBetaEstimator()
bmom = OnlineBetaMOM()
for p in p_values[np.random.permutation(p_values.shape[0])]:    
    bbe.update(p)
    bmom.update(p)
print(bbe.get_params())
print(bmom.get_params())

(0.8265861237991307, 1.1734138762008692)
(0.9123651518970913, 1.296729835669172)


In [74]:
beta.fit(p_values, loc=0, scale=1)

(0.9339987845749813,
 1.3030315412931242,
 0.00040363596282244613,
 1.0030119440994247)